In [0]:
%run ./streeming_word_count

In [0]:
class WcTestSuite():
    def __init__(self,base_dir):
        self.base_dir = base_dir

    def cleanupTests(self):
        print("cleanup started...")
        spark.sql("drop table if exists streeming.demo_db.word_count")
        dbutils.fs.rm(f"{self.base_dir}/data/text",True)
        dbutils.fs.mkdirs(f"{self.base_dir}/data/text")
        print("done")
        
    
    def ingest_iteration(self,iter):
        print("ingestion started...")
        dbutils.fs.cp(f"{self.base_dir}/ingestion/text_data_{iter}.txt",f"{self.base_dir}/data/text",True)
        print("done")

    def run_test(self,expected_count):
        print("Test started...")
        actual_count = spark.sql("select sum(count) from streeming.demo_db.word_count where substr(word,1,1) = 's'").collect()[0][0]
        assert actual_count == expected_count, f"Test failed actual count is {actual_count} "
        print("done")

    def run_test_suite(self): 
        wc = BatchWC(self.base_dir)

        print("testing iteration 1")
        self.cleanupTests()
        self.ingest_iteration(1)
        wc.word_count()
        self.run_test(25)
        print("first iteration batch word count completed")

        print("testing iteration 2")
        self.cleanupTests()
        self.ingest_iteration(2)
        wc.word_count()
        self.run_test(7)
        print("second iteration batch word count completed")

        print("testing iteration 3")
        self.cleanupTests()
        self.ingest_iteration(3)
        wc.word_count()
        self.run_test(5)
        print("third iteration batch word count completed")
        
    
        

    


In [0]:
test = WcTestSuite('/Volumes/streeming/demo_db/datasource')
test.run_test_suite()